In [1]:
# assoc_feed
# Does the pre-processing of arrivals to feed to the associator

In [2]:
import numpy as np
import pandas as pd
import os
import json
import requests

from flask import Flask, request, jsonify

In [3]:
# Import configuration paramaters for pipeline
import pconf

In [4]:
# App variables
app = Flask(__name__)
app_name = 'assoc_feed'
napp = 'assoc'
duplicate_threshold = 0.25
# arrival_count = 0
# assoc_bs = 20
# assoc_buffer = []

In [5]:
# Test up method
@app.route('/')
def apitest():
    return f'{app_name} is working'

In [6]:
def clean_catalog(filename):
    #Generate Dataframe from CSV File, apply Datetime format to date/times.
    cat = pd.read_csv(filename)
    cat.columns = ['ARID','PHASE','DATETIME','STA','STA_LAT','STA_LON']
    cat['DATETIME'] = pd.to_datetime(cat['DATETIME'])
    
    # Sort in place
    cat.sort_values(by=['STA', 'DATETIME', 'PHASE'], ignore_index=True, inplace=True)

    #Create a copy of cat and shift it down by one row
    shiftcat = cat.copy(deep = True)
    shiftcat.drop(labels=0, axis=0, inplace = True)
    shiftcat.reset_index(drop = True, inplace = True)

    #Compare DATETIME from cat to shiftcat DATETIME, generate timedeltas.
    cat['TIMEDELT'] = (shiftcat.DATETIME - cat.DATETIME).dt.total_seconds().abs()
    cat['SHIFTPHASE'] = shiftcat.PHASE
    cat['SHIFTSTA'] = shiftcat.STA
    
    #Creates a new dataframe that removes any picks that were within .25 seconds of another pick of the same phase occuring on the same station.
    cat = cat[~((cat['TIMEDELT'] <= duplicate_threshold) & (cat['PHASE'] == cat['SHIFTPHASE']) & (cat['STA'] == cat['SHIFTSTA']))]
    cat.drop(columns = ['TIMEDELT','SHIFTPHASE','SHIFTSTA'], inplace=True)
    cat.reset_index(drop = True, inplace=True)

    return cat

In [7]:
# This is the URI that process the received requests
@app.route(f'/{app_name}', methods=['POST'])
def process_request():  
    # Read the catalog filename from the pdict
    pdict = request.json
    cat = pdict['cat']
    
    # Clean the catalog of duplicates
    cleancat = clean_catalog(cat)
    clean_cat = pdict['clean_cat']
    
    cleancat.to_csv(clean_cat, index=False)
    
    assoc_response = forward(pdict)
    
    return ('', 204)

In [8]:
def forward(json_data):
    response = requests.post(f'http://{pconf.host}:{pconf.apps[napp]}/{napp}', json = json_data, headers = pconf.head)
    return ('', 204)

In [ ]:
if __name__ == '__main__':
    app.run(pconf.host, debug=False, port=pconf.apps[app_name])

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6004/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2021 22:59:34] "POST /assoc_feed HTTP/1.1" 204 -
127.0.0.1 - - [10/Nov/2021 23:04:09] "POST /assoc_feed HTTP/1.1" 204 -
